In [1]:
from torchvision import datasets, transforms
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
transform=transforms.Compose([transforms.ToTensor()])

In [3]:
MNIST = datasets.MNIST('D:/', train= True, download = False, transform = transform)

In [4]:
train_loader = DataLoader(MNIST, batch_size=32, shuffle=True)

In [5]:
class MNIST_NET(nn.Module):
    def __init__(self):
        super(MNIST_NET, self).__init__()
        self.linear1 = nn.Linear(28*28,500)
        self.linear2 = nn.Linear(500,10)
    def forward(self, x):
        x =F.relu(self.linear1(x))
    
        return F.softmax(self.linear2(x), dim = 1)


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else'cpu')
model = MNIST_NET().to(device)
#model.load_state_dict(torch.load('Saved_Weights/ANN_11.pt'))

In [7]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr = 0.001)

In [16]:
start_epoch = 0
end_epoch = 10
for epoch in range(start_epoch, end_epoch):
    total_loss = 0
    for idx , (image,label) in enumerate(train_loader):
        model.zero_grad()
        inp = image.to(device).float()
        inp = inp.view(-1, 28*28) #flattening of image
        output = model(inp.to(device))
           
        loss = criterion(output, label.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print('Epoch {}/{} , Loss: {}'.format(epoch + 1, end_epoch , total_loss/len(train_loader)))
    torch.save(model.state_dict(), 'Saved_Weights/ANN_{}.pt'.format(epoch))

Epoch 1/10 , Loss: 1.5810901753107707
Epoch 2/10 , Loss: 1.5073799001057944
Epoch 3/10 , Loss: 1.4943492153803508
Epoch 4/10 , Loss: 1.488134237353007
Epoch 5/10 , Loss: 1.4839426522572836
Epoch 6/10 , Loss: 1.4803729724248251
Epoch 7/10 , Loss: 1.478089557647705
Epoch 8/10 , Loss: 1.4761463452021282
Epoch 9/10 , Loss: 1.474911158879598
Epoch 10/10 , Loss: 1.4737694730758668


In [8]:
test_dataset = datasets.MNIST('D:/',train= False, download = False, transform = transform)

In [18]:
test_loader = DataLoader (test_dataset , batch_size= 1)

In [20]:
accuracy = 0
total_loss = 0
with torch.no_grad():
    for idx, (image , labels) in enumerate (test_loader):
         
        inp = image.to(device).float()
        output = model(inp.view(-1, 28*28)) #flatten image as input
        pred = output.argmax(dim= 1 , keepdim = True)
        #print(pred.item())
        total_loss += criterion(output, labels.to(device)).item()
        if pred == labels.item():
            accuracy +=1
        #print(".................................................................")
    print('Accuracy: ',accuracy/len(test_loader)* 100 , 'Loss: ',total_loss/len(test_loader))


Accuracy:  97.8 Loss:  1.4834744889616966
